In [19]:
bits = 4
def f(A, B, C, D): return ~A*B+~C*D  # Função a ser calculada

# Classe Bit


In [20]:
class Bit:
    def __init__(self, bitvalue):
        if bitvalue not in [0, 1, "x"]:
            raise ValueError(
                "Os possíveis valores para o argumento 'bitvalue' são 0, 1 e 'x'.")
                
        self._value = bitvalue

    def __bool__(self):
        if self._value == "x":
            return True
        return self._value == 1

    def __invert__(self):
        if self._value == 1:
            return Bit(0)
        if self._value == 0:
            return Bit(1)
        return Bit("x")

    def __sub__(self, other):
        # self and not other
        if isinstance(other, bool):
            other = Bit(int(other))
        return self.__and__(~other)

    def __or__(self, other):
        if self._value == 1 or other._value == 1:
            return Bit(1)
        elif self._value == 0:
            return Bit(other._value)
        elif other._value == 0:
            return Bit(self._value)
        return Bit("x")
    __add__ = __or__

    def __and__(self, other):
        if isinstance(other, bool):
            other = Bit(int(other))
        if self._value == 0 or other._value == 0:
            return Bit(0)
        elif self._value == 1:
            return Bit(other._value)
        elif other._value == 1:
            return Bit(self._value)
        return Bit("x")

    __mul__ = __and__

    def __xor__(self, other):
        if isinstance(other, bool):
            other = Bit(int(other))
        if self._value == "x" or other._value == "x":
            return Bit("x")
        elif self._value == other._value:
            return Bit(0)
        return Bit(1)

    def __str__(self):
        return str(self._value)

    def __repr__(self):
        return f"{self._value}"

    def __int__(self):
        return self.value

    def __eq__(self, other):
        if isinstance(other, bool):
            other = Bit(int(other))
        return self._value == other._value

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, bitvalue):
        if bitvalue in [0, 1, "x"]:
            self._value = bitvalue
            return
        raise ValueError("Os possíveis valores para 'value' são 0, 1 e 'x'.")


a = Bit(1)
b = Bit(0)
c = Bit("x")
print(a+b)
print(a or b)
print(a*b and c)
print(a ^ b)

1
1
0
1


# Código


In [22]:
import itertools
import string
from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl import styles
from itertools import permutations, chain
import pandas as pd
import numpy as np

nome = "testeand.xlsx"  # Nome do arquivo gerado em excel


def imprimematriz(mat, espaco=5):  # *Mostra* a matriz formatada
    for i in range(len(mat)):  # Percorre as linhas
        # print(i + 1, end=' ')  # Printa o número da linha
        for j in range(len(mat[i])):  # Percorre cada termo da linha
            if j < len(mat[i]) - 1:  # Enquanto j n for o ultimo termo da linha
                print(f'{mat[i][j]:<{espaco}}', end=' ')
            else:
                print(f'{mat[i][j]}')


def to_binario(decimal):
    bitsList = []

    while decimal > 0:
        bitsList = [decimal % 2] + bitsList
        decimal //= 2

    if len(bitsList) == 0:
        return [0]

    return bitsList


values = [Bit(0), Bit(1)]
# Matriz com as possibilidades de bits
m = list(itertools.product((0, 1), repeat=bits))
r = list()  # Vetor Respostas

# Vetor Resultado
for item in m:
    list_bit = list(map(Bit, item))
    r.append([list_bit, f(*list_bit)])

r = pd.DataFrame(r)

r2 = r[0].apply(pd.Series)

result = pd.concat((r2, r[1]), axis=1)
result.columns = list(string.ascii_lowercase)[:r2.shape[1]]+['S']
result.index = ['']*len(result)
result.columns = 'T0,T1,C0,C1,E'.split(',')
result.style.highlight_max()
result

,T0,T1,C0,C1,E
,0,0,0,0,0
,0,0,0,1,1
,0,0,1,0,0
,0,0,1,1,0
,0,1,0,0,1
,0,1,0,1,1
,0,1,1,0,1
,0,1,1,1,1
,1,0,0,0,0
,1,0,0,1,1


# Karnaught


In [ ]:
mapa = result.S.to_numpy().reshape(4, 4).copy()
mapa[:, [-1, -2]] = mapa[:, [-2, -1]].copy()
mapa[[-1, -2], :] = mapa[[-2, -1], :].copy()

gray = ['00', '01', '11', '10']
mapa = pd.DataFrame(mapa, columns=gray, index=gray)
mapa

,00,01,11,10
00,0,0,0,0
01,0,0,0,0
11,0,1,1,1
10,1,1,1,1
